In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from xml.etree.ElementTree import parse

In [8]:
label_number_dict = {'cat':0, 'dog':1}

def xml_read(xml_path):
    root = parse(xml_path).getroot()
    image_info = root.findall("image")
    
    for image in image_info:
        bbox = image.findall('box')
        
        image_width = image.attrib['width']
        image_height = image.attrib['height']
        
        image_name = image.attrib['name']
        image_name_temp = image_name
        image_name = os.path.join("./", image_name)
        
        # image read & 변환
        image = cv2.imread(image_name)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        for box_info in bbox:
            label = box_info.attrib['label']
            
            # 값 -> string
            xtl = box_info.attrib['xtl']
            ytl = box_info.attrib['ytl']
            xbr = box_info.attrib['xbr']
            ybr = box_info.attrib['ybr']
            
            # int -> float 변환
            # 소수점이 포함이라 바로 int값 변환불가
            
            xtl_f = float(xtl)
            ytl_f = float(ytl)            
            xbr_f = float(xbr)            
            ybr_f = float(ybr)
            
            # float -> int 변환
            x1_i = int(xtl_f)
            y1_i = int(ytl_f)
            x2_i = int(xbr_f)
            y2_i = int(ybr_f)
            
            # image_width, image_height 경우, string -> int 형변환 필요
            img_width_i = int(image_width)
            img_height_i = int(image_height)
            
            # Pascal_xyxy to YOLO center_x, center_y, yolo_w, yolo_h
            center_x = ((x2_i + x1_i)/(2*img_width_i))
            center_y = ((y2_i + y1_i)/(2*img_height_i))
            yolo_w = (x2_i - x1_i)/img_width_i
            yolo_h = (y2_i - y1_i)/img_height_i
            
            # file_name
            image_name_temp = image_name_temp.replace('.jpg','')
            
            # label cat, dog -> 0,1로 변경
            label_number = label_number_dict[label]
            print("Pascal_xyxy to YOLO >> ", label_number, center_x, center_y, yolo_w, yolo_h)
            
            # 텍스트로 저장
            with open(f'./label_data/{image_name_temp}_xml.txt','a') as f :
                f.write(f"{label_number} {center_x} {center_y} {yolo_w} {yolo_h} \n")

if __name__ == "__main__":                
    xml_read("./annotations.xml")

Pascal_xyxy to YOLO >>  1 0.865625 0.45 0.26875 0.5166666666666667
Pascal_xyxy to YOLO >>  0 0.16171875 0.6041666666666666 0.3140625 0.44583333333333336
